In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import gseapy as gp
import logging

In [3]:
gp.__version__

'0.9.0'

In [15]:
def enrichment_score_tensor(gene_mat, cor_mat, gene_sets, weighted_score_type, nperm=1000,
                            scale=False, single=False, rs=np.random.RandomState()):
    """Next generation algorithm of GSEA and ssGSEA.

    :param gene_mat:        the ordered gene list(vector) or gene matrix.
    :param cor_mat:         correlation vector or matrix  (e.g. signal to noise scores)
                            corresponding to the genes in the gene list or matrix.
    :param dict gene_sets:  gmt file dict.
    :param float weighted_score_type:     weighting by the correlation.
                            options: 0(classic), 1, 1.5, 2. default:1 for GSEA and 0.25 for ssGSEA.
    :param int nperm:       permutation times.
    :param bool scale:      If True, normalize the scores by number of genes_mat.
    :param bool single:     If True, use ssGSEA algorithm, otherwise use GSEA.
    :param rs:              Random state for initialize gene list shuffling.
                            Default: np.random.RandomState(seed=None)
    :return:
             ES: Enrichment score (real number between -1 and +1), it's true for ssGSEA, only scaled

             ESNULL: Enrichment score calcualted from random permutation

             Hits_Indices: Indices of genes if genes are included in gene_set.

             RES: Numerical vector containing the running enrichment score for
                  all locations in the gene list .

    """
    # gene_mat -> 1d: prerank, ssSSEA or 2d: GSEA
    keys = sorted(gene_sets.keys())

    if weighted_score_type == 0:
        # don't bother doing calcuation, just set to 1
        cor_mat = np.ones(cor_mat.shape)
    elif weighted_score_type > 0:
        pass
    else:
        logging.error("Using negative values of weighted_score_type, not allowed")
        sys.exit(0)

    cor_mat = np.abs(cor_mat)
    if cor_mat.ndim ==1:
        # ssGSEA or Prerank
        #genestes->M, genes->N, perm-> axis=2
        N, M = len(gene_mat), len(keys)
        # generate gene hits matrix
        # for 1d ndarray of gene_mat, set assume_unique=True,
        # means the input arrays are both assumed to be unique,
        # which can speed up the calculation.
        tag_indicator = np.vstack([np.in1d(gene_mat, gene_sets[key], assume_unique=True) for key in keys])
        # index of hits
        hit_ind = [ np.flatnonzero(tag).tolist() for tag in tag_indicator ]
        # generate permutated hits matrix
        perm_tag_tensor = np.repeat(tag_indicator, nperm+1).reshape((M,N,nperm+1))
        # shuffle matrix, last matrix is not shuffled
        np.apply_along_axis(lambda x: np.apply_along_axis(rs.shuffle,0,x),1, perm_tag_tensor[:,:,:-1])
        # missing hits
        no_tag_tensor = 1 - perm_tag_tensor
        # calculate numerator, denominator of each gene hits
        rank_alpha = (perm_tag_tensor*cor_mat[np.newaxis,:,np.newaxis])** weighted_score_type

    elif cor_mat.ndim == 2:
        # GSEA
        # 2d ndarray, gene_mat and cor_mat are shuffled already
        # reshape matrix
        cor_mat, gene_mat = cor_mat.T, gene_mat.T
        # genestes->M, genes->N, perm-> axis=2
        # don't use assume_unique=True in 2d array when use np.isin().
        # elements in gene_mat are not unique, or will cause unwanted results
        perm_tag_tensor = np.stack([np.isin(gene_mat, gene_sets[key]) for key in keys], axis=0)
        #index of hits
        hit_ind = [ np.flatnonzero(tag).tolist() for tag in perm_tag_tensor[:,:,-1] ]
        # nohits
        no_tag_tensor = 1 - perm_tag_tensor
        # calculate numerator, denominator of each gene hits
        rank_alpha = (perm_tag_tensor*cor_mat[np.newaxis,:,:])** weighted_score_type
    else:
        logging.error("Program die because of unsupported input")
        sys.exit(0)

    # Nhint = tag_indicator.sum(1)
    # Nmiss =  N - Nhint
    axis=1
    P_GW_denominator = np.sum(rank_alpha, axis=axis, keepdims=True)
    P_NG_denominator = np.sum(no_tag_tensor, axis=axis, keepdims=True)
    REStensor = np.cumsum(rank_alpha / P_GW_denominator - no_tag_tensor / P_NG_denominator, axis=axis)
    # ssGSEA: scale es by gene numbers ?
    # https://gist.github.com/gaoce/39e0907146c752c127728ad74e123b33
    if scale: REStensor = REStensor / len(gene_mat)
    if single:
        #ssGSEA
        esmatrix = np.sum(REStensor, axis=axis)
    else:
        #GSEA
        esmax, esmin = REStensor.max(axis=axis), REStensor.min(axis=axis)
        esmatrix = np.where(np.abs(esmax)>np.abs(esmin), esmax, esmin)

    es, esnull, RES = esmatrix[:,-1], esmatrix[:,:-1], REStensor[:,:,-1]

    return es, esnull, hit_ind, RES


In [16]:
gex = pd.read_table("./data/P53_resampling_data2.txt", index_col=0)

In [17]:
gex.head()

,786-0,BT-549,CCRF-CEM,COLO 205,EKVX,HCC-2998,HCT-15,HOP-62,HOP-92,HS 578T,...,MCF7,MOLT-4,NCI-H460,OVCAR-4,SF-539,SK-MEL-5,SR,UACC-257,UACC-62,UO-31
NAME,,,,,,,,,,,,,,,,,,,,,
CTLA2B,111.19,86.22,121.85,75.19,208.62,130.59,124.72,324.09,242.71,89.71,...,163.76,59.50,134.12,152.09,197.46,137.79,81.53,123.37,81.41,180.78
SCARA3,460.30,558.34,183.55,37.29,158.00,43.61,80.83,300.08,1250.25,144.82,...,109.91,120.42,73.06,115.03,95.12,37.56,76.16,41.10,77.51,519.17
LOC100044683,97.25,118.94,81.17,119.51,119.88,107.73,165.57,203.97,135.43,171.09,...,222.84,124.98,114.75,141.66,170.19,147.70,157.48,152.18,98.89,118.06
CMBL,33.45,55.10,221.67,50.30,35.12,75.70,84.01,44.12,79.96,29.01,...,51.32,117.11,59.46,78.46,45.55,49.07,96.69,33.09,10.38,52.89
CLIC6,35.75,41.26,63.04,219.86,42.53,54.19,86.98,71.20,53.89,98.86,...,154.05,31.62,37.66,32.64,63.35,27.95,70.99,36.25,17.50,49.41


In [18]:
gmt = gp.parser.gsea_gmt_parser("./data/randomSets.gmt")

In [19]:
gmt.keys()

dict_keys(['random1', 'random2', 'random3', 'random4', 'random5', 'level2_rand', 'level4_rand', 'level6_rand', 'level8_rand', 'level10_RAND', 'level12_random'])

In [20]:
gm = gmt['random2']

In [21]:
print(gm)

['GRB14', 'KAZALD1', 'POLR2I', 'C7orf26', 'MYOZ3', 'CRYBA4', 'C9orf85', 'PRPS1', 'C9', 'GTF2H4', 'PSME2', 'HAUS4', 'VPS16', 'SCOC', 'RHAG', 'AIF1', 'RPL41', 'C16orf5', 'LCT', 'C1orf83', 'GFAP', 'NUDCD3', 'ROGDI', 'HEATR1', 'MST1R', 'ZMPSTE24', 'HDAC1', 'NEO1', 'POLR3A', 'VPS54', 'F5', 'QKI', 'ITFG2', 'PPP2R3A', 'LIMS2', 'PCDH15', 'STOML2', 'FLT3', 'GABRR1', 'GNPDA2', 'PHLDA3', 'RARS', 'MRPS33', 'LCK', 'PTN', 'HRG', 'EIF3I', 'PMVK', 'UBOX5', 'VN2R1P', 'STAP2', 'CCNB3', 'ADAM8', 'LHCGR', 'PERP', 'COL1A2', 'ZSWIM1', 'BCAP29', 'PTP4A3', 'PIP4K2A', 'PRRX2', 'UHRF1', 'CEBPZ', 'UBE2J1', 'WFDC2', 'SGK2', 'ZBED3', 'CCDC82', 'TMOD1', 'CD2AP', 'C6orf203', 'TMEM85']


In [24]:
names=[]
es = []
hits = []
rs = np.random.RandomState(0)
gexrnk = gex.rank(axis=0, method='average', na_option='bottom')
for name, ser in gexrnk.iteritems():
    ser = ser.sort_values(ascending=False)
    est = enrichment_score_tensor(gene_mat=ser.index.values, cor_mat=ser.values, gene_sets={'random2':gm},
                                  weighted_score_type=0.25, nperm=10, 
                                  scale=True, single=True, rs=rs)[0]
    es.append(est)
    names.append(name)

In [25]:
for n, e  in zip(names, es):
    print(n, ": ", e)

786-0 :  [ 0.1760403]
BT-549 :  [ 0.16091362]
CCRF-CEM :  [ 0.14796385]
COLO 205 :  [ 0.17646881]
EKVX :  [ 0.14666306]
HCC-2998 :  [ 0.15656921]
HCT-15 :  [ 0.14082521]
HOP-62 :  [ 0.17807567]
HOP-92 :  [ 0.20020026]
HS 578T :  [ 0.14616472]
HT29 :  [ 0.13500038]
K-562 :  [ 0.15385671]
KM12 :  [ 0.12565788]
M14 :  [ 0.22820931]
MDA-MB-231/ATCC :  [ 0.15009001]
MDA-MB-435 :  [ 0.19923792]
NCI-H23 :  [ 0.17022116]
NCI-H322M :  [ 0.14975917]
NCI-H522 :  [ 0.16470175]
OVCAR-3 :  [ 0.12186676]
OVCAR-5 :  [ 0.09593231]
OVCAR-8 :  [ 0.18930624]
PC-3 :  [ 0.00528293]
RXF-393 :  [ 0.12740607]
SF-268 :  [ 0.14659735]
SF-295 :  [ 0.218491]
SK-MEL-2 :  [ 0.21369779]
SN12C :  [ 0.14884469]
SNB-19 :  [ 0.1903936]
SNB-75 :  [ 0.19304376]
SW-620 :  [ 0.12421174]
T-47D :  [ 0.21318539]
U251 :  [ 0.1296239]
A498 :  [ 0.14378098]
A549/ATCC :  [ 0.15998232]
ACHN :  [ 0.15651666]
CAKI-1 :  [ 0.13959061]
HCT-116 :  [ 0.16013915]
LOX IMVI :  [ 0.1926479]
MALME-3M :  [ 0.2153658]
MCF7 :  [ 0.14457417]
MOLT-4

In [26]:
## no scale es
names=[]
es = []
hits = []
rs = np.random.RandomState(0)
gexrnk = gex.rank(axis=0, method='average', na_option='bottom')
for name, ser in gexrnk.iteritems():
    ser = ser.sort_values(ascending=False)
    est = enrichment_score_tensor(gene_mat=ser.index.values, cor_mat=ser.values, gene_sets={'random2':gm},
                                  weighted_score_type=0.25, nperm=100, 
                                  scale=True, single=True, rs=rs)[0]
    es.append(est)
    names.append(name)

In [27]:
## no scale es values and norm by all samples
es = np.array(es)
nes = es/(es.max() -es.min())
for n, e  in zip(names, nes):
    print(n, ": ", e)

786-0 :  [ 0.78967904]
BT-549 :  [ 0.721824]
CCRF-CEM :  [ 0.66373414]
COLO 205 :  [ 0.79160126]
EKVX :  [ 0.65789907]
HCC-2998 :  [ 0.70233593]
HCT-15 :  [ 0.63171171]
HOP-62 :  [ 0.79880929]
HOP-92 :  [ 0.89805545]
HS 578T :  [ 0.65566362]
HT29 :  [ 0.60558278]
K-562 :  [ 0.69016823]
KM12 :  [ 0.56367432]
M14 :  [ 1.02369807]
MDA-MB-231/ATCC :  [ 0.67327162]
MDA-MB-435 :  [ 0.89373859]
NCI-H23 :  [ 0.76357563]
NCI-H322M :  [ 0.67178754]
NCI-H522 :  [ 0.73881675]
OVCAR-3 :  [ 0.54666816]
OVCAR-5 :  [ 0.43033177]
OVCAR-8 :  [ 0.84918723]
PC-3 :  [ 0.02369807]
RXF-393 :  [ 0.57151632]
SF-268 :  [ 0.65760428]
SF-295 :  [ 0.9801038]
SK-MEL-2 :  [ 0.95860248]
SN12C :  [ 0.66768539]
SNB-19 :  [ 0.85406488]
SNB-75 :  [ 0.86595292]
SW-620 :  [ 0.55718725]
T-47D :  [ 0.95630399]
U251 :  [ 0.58146503]
A498 :  [ 0.64497066]
A549/ATCC :  [ 0.71764641]
ACHN :  [ 0.70210021]
CAKI-1 :  [ 0.62617357]
HCT-116 :  [ 0.71834993]
LOX IMVI :  [ 0.86417721]
MALME-3M :  [ 0.96608483]
MCF7 :  [ 0.64852875]
MO